# 環境を操作する


Azure Machine Learning ジョブとしてスクリプトを実行する場合は、ジョブ実行の実行コンテキストを定義する必要があります。 主要な構成の 1 つは、スクリプトを実行するコンピューティング先です。 これは、ローカル ワークステーション (この場合はコンピューティング インスタンス)、またはリモート コンピューティング ターゲット (オンデマンドでプロビジョニングされる Azure Machine Learning マネージド コンピューティング クラスターなど) です。

このノートブックでは、コンピューティング クラスターを作成し、ジョブのコンピューティング ターゲットを確認します。

## 開始する前に

このノートブックでコードを実行するには、最新バージョンの **azureml-ai-ml** パッケージが必要です。 次のセルを実行して、パッケージがインストールされていることを確認します。

> **注**:
> **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールします。

In [ ]:
## ワークスペースに接続する

必要な SDK パッケージがインストールされているため、ワークスペースに接続できます。

ワークスペースに接続するには、識別子パラメーター (サブスクリプション ID、リソース グループ名、ワークスペース名) が必要です。 リソース グループ名とワークスペース名は既に入力されています。 コマンドを完了するには、サブスクリプション ID のみが必要です。

必要なパラメーターを見つけるには、Studio の右上にあるサブスクリプションとワークスペース名をクリックします。 右側にペインが開きます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> サブスクリプション ID をコピーし、**YOUR-SUBSCRIPTION-ID** をコピーした値に置き換えます。 </p>

## スクリプトをジョブとして実行する

モデルをトレーニングするには、まず **src** フォルダーに **diabetes_training.py** スクリプトを作成します。 このスクリプトでは、トレーニング データと同じフォルダー内の **diabetes.csv** ファイルが使用されます。

スクリプトの先頭にライブラリをインポートします。 これらのライブラリの関数は、データの処理とモデルのトレーニングに使用されます。 スクリプトの実行に使用するコンピューティングには、これらのライブラリがインストールされている必要があります。

In [ ]:
スクリプトを作成したら、スクリプトをジョブとして実行できます。 このスクリプトでは、一般的なライブラリが使用されます。 そのため、pandas、numpy、scikit-learn などを含むキュレーション済みの環境を使用できます。

ジョブでは、キュレーション済みの環境の最新バージョン `AzureML-sklearn-0.24-ubuntu18.04-py37-cpu` が使用されます。

In [ ]:
ジョブの実行中でも、次のセルを実行できます。

## 環境の一覧表示

ワークスペース内の環境を確認しましょう。 

前のジョブでは、キュレーション済みの環境の 1 つを使用しました。 ワークスペースに既に存在するすべての環境を確認するには、環境を一覧表示します。 

In [ ]:
すべての選別された環境には、**AzureML-** で始まる名前が付いています (このプレフィックスは、独自の環境では使用できません)。

特定の環境を確認するには、その名前とバージョンで環境を取得します。 たとえば、前のジョブで使用したキュレーション済みの環境の*説明*と*タグ*を取得できます。

## カスタム環境を作成して使用する

キュレーション済みの環境に、スクリプトを実行するために必要なすべての Python パッケージが含まれない場合は、独自のカスタム環境を作成できます。 環境に必要なすべてのパッケージを一覧表示することで、スクリプトを簡単に再実行できます。 すべての依存関係は、使用するコンピューティングに関係なく、ジョブ構成で指定できる環境に格納されます。

たとえば、Docker イメージから単純に環境を作成できます。 PyTorch などの特定のフレームワークには、必要なものがすべて既に含まれるパブリック Docker イメージがあります。 

Docker イメージから環境を作成しましょう。

In [ ]:
これで、環境がワークスペースに登録され、スクリプトをジョブとして実行するときに参照できます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> ジョブはすぐに失敗します。 エラー メッセージを確認します。 </p>

エラー メッセージは、pandas という名前のモジュールがないことを示します。 このエラーには、考えられる原因が 2 つあります。

- スクリプトでは pandas が使用されますが、ライブラリがインポートされませんでした (`import pandas as pd`)。 
- スクリプトでは、スクリプトの上部にあるライブラリがインポートされますが、コンピューティングにライブラリがインストールされていませんでした (`pip install pandas`)。

`diabetes-training.py` スクリプトを確認した後、スクリプトが正しいことを確認できます。これは、ライブラリがインストールされていないことを意味します。 つまり、環境に必要なパッケージが含まれていませんでした。

前のジョブで使用した基本 Docker イメージを使用して、新しい環境を作成してみましょう。 ここで、conda 仕様を追加して、必要なパッケージがインストールされるようにします。 まず、次のセルを実行して conda 仕様ファイルを作成します。

スクリプトを正常に実行するために必要な依存関係がすべて conda 仕様ファイルに含まれています。

基本 Docker イメージ**と** conda 仕様ファイルを使用して新しい環境を作成し、必要な依存関係を追加します。 Azure Machine Learning により、提供された Docker イメージを基盤にして conda 環境がビルドされます。 

In [ ]:
これで、新しい環境でジョブを送信してスクリプトを実行できるようになりました。

新しいカスタム環境でジョブを送信すると、環境のビルドがトリガーされます。 新しく作成された環境を初めて使用するときは、環境のビルドに 10 分から 15 分かかる場合があります。これは、ジョブの完了にも時間がかかることを意味します。 

ジョブを送信する前に、環境のビルドを手動でトリガーすることもできます。 環境は、初めて使用する場合にのみビルドする必要があります。 

ジョブによって新しい環境のビルドがトリガーされると、ジョブの **[出力とログ]** タブでビルドのログを確認できます。 **azureml-logs/20_image_build_log.txt** を開いて、環境ビルドのログを調べます。 

![ビルド ログのスクリーンショット](./images/screenshot-logs.png)

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env. description, env.tags)

## Create and use a custom environment

If a curated environment doesn't include all the Python packages you need to run your script, you can create your own custom environment. By listing all necessary packages in an environment, you can easily re-run your scripts. All the dependencies are stored in the environment which you can then specify in the job configuration, independent of the compute you use.

For example, you can create an environment simply from a Docker image. Certain frameworks like PyTorch will have a public Docker image that already includes everything you need. 

Let's create an environment from a Docker image:

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

The environment is now registered in your workspace and you can reference it when you run a script as a job:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> The job will quickly fail! Review the error message. </p>

The error message will tell you that there is no module named pandas. There are two possible causes for such an error:

- The script uses pandas but didn't import the library (`import pandas as pd`). 
- The script does import the library at the top of the script but the compute didn't have the library installed (`pip install pandas`).

After reviewing the `diabetes-training.py` script you can observe the script is correct, which means the library wasn't installed. In other words, the environment didn't include the necessary packages.

Let's create a new environment, using the base Docker image used in the previous job. Now, you'll add a conda specification to ensure the necessary packages will be installed. First, run the following cell to create the conda specification file:

In [ ]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Note that all necessary dependencies are included in the conda specification file for the script to run successfully.

Create a new environment using the base Docker image **and** the conda specification file to add the necessary dependencies. Azure Machine Learning will build the conda environment on top of the Docker image you provided. 

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Now, you can submit a job with the new environment to run the script:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-plus-conda-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Submitting the job with the new custom environment triggers the build of the environment. The first time you use a newly created environment, it can take 10-15 minutes to build the environment, which also means your job will take longer to complete. 

You can also choose to manually trigger the build of the environment before you submit a job. The environment only needs to be built the first time you use it. 

When the job triggers the build of a new environment, you can review the logs of the build in the **Outputs + logs** tab of the job. Open **azureml-logs/20_image_build_log.txt** to inspect the logs of the environment build. 

![Screenshot build logs](./images/screenshot-logs.png)